activate conda environment sklearn.yml

## IMPORTS

In [1]:
import numpy as np
import pandas as pd
from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay

from numpy.random import default_rng
rng = default_rng()

from pygel3d import hmesh

## FUNCTIONS

In [2]:
def hull_dist(hull, positions):
    m = hmesh.Manifold().from_triangles(hull.points, hull.simplices)
        
    dist = hmesh.MeshDistance(m)
    result = dist.signed_distance(positions)
    return(np.absolute(result))
    

In [3]:
def in_hull(hull, p):

    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

In [5]:
def hull_plot(gampos, gamhull, ngampos):
    plt.scatter(gampos[:,0], gampos[:,2])
    plt.scatter(gampos[gamhull.vertices,0], gampos[gamhull.vertices,2])
    plt.scatter(ngampos[:,0], ngampos[:,2], alpha=.5)
    plt.show()

## Assignments

In [7]:
hulls = pd.read_csv("convex_hulls_v1.csv")
hulls.head()

mean1      mean2      mean3            species
0  -84.875370  -4.078943 -18.389744  Anopheles_gambiae
1  -92.972690 -11.120659 -16.512430  Anopheles_gambiae
2  -91.415210  -5.447519 -19.713837  Anopheles_gambiae
3 -101.515625 -56.799920  14.513015  Anopheles_gambiae
4  -82.447190  -5.085171 -17.396854  Anopheles_gambiae

In [8]:
gampos = hulls.loc[hulls.species=='Anopheles_gambiae', ['mean1', 'mean2', 'mean3']].values
gamhull = ConvexHull(gampos)
colpos = hulls.loc[hulls.species=='Anopheles_coluzzii', ['mean1', 'mean2', 'mean3']].values
colhull = ConvexHull(colpos)
arapos = hulls.loc[hulls.species=='Anopheles_arabiensis', ['mean1', 'mean2', 'mean3']].values
arahull = ConvexHull(arapos)
tenpos = hulls.loc[hulls.species=='Anopheles_tengrela', ['mean1', 'mean2', 'mean3']].values
tenhull = ConvexHull(tenpos)
melpos = hulls.loc[hulls.species=='Anopheles_melas', ['mean1', 'mean2', 'mean3']].values
melhull = ConvexHull(melpos)
merpos = hulls.loc[hulls.species=='Anopheles_merus', ['mean1', 'mean2', 'mean3']].values
merhull = ConvexHull(merpos)
quadpos = hulls.loc[hulls.species=='Anopheles_quadriannulatus', ['mean1', 'mean2', 'mean3']].values
quadhull = ConvexHull(quadpos)
bwapos = hulls.loc[hulls.species=='Anopheles_bwambae_fontenillei', ['mean1', 'mean2', 'mean3']].values
bwahull = ConvexHull(bwapos)

In [9]:
#Check that the distance function works
#Should be values 6.30, 13.03, 6.57, 68.12, 7.74

test_hull_distance = np.zeros(gampos.shape[0])
for i in np.arange(len(test_hull_distance)):
    test_hull_distance[i] = hull_dist(colhull, gampos[i])
df_test_hull_distance = pd.DataFrame(test_hull_distance, columns=['dist_to_coll_hull'])
df_test_hull_distance.head()

dist_to_coll_hull
0           6.309639
1          13.038807
2           6.575978
3          68.123405
4           7.745610

In [ ]:
#Import test data
testdir = 
val = pd.read_csv(testdir+'gambiae_complex/latent_coords.csv')
print(val.shape)
#only samples with at least 50 amplicons
val = val.loc[(val.amplified>49)].reset_index()
print(val.shape)
valX = val[['mean1', 'mean2', 'mean3']].values

In [52]:
#First round, assign points inside hulls
for label, hull in zip(['Anopheles_gambiae', 'Anopheles_coluzzii', 'Anopheles_arabiensis', 'Anopheles_tengrela', 'Anopheles_merus', 'Anopheles_melas', 
                                'Anopheles_quadriannulatus', 'Anopheles_bwambae-fontenillei'], 
                               [gampos, colpos, arapos, tenpos, merpos, melpos, quadpos, bwapos]):
    val.loc[in_hull(hull, val[['mean1', 'mean2', 'mean3']].values), 'assigned_species'] = label

In [ ]:
val.groupby('assigned_species', dropna=False)['sample_id'].count()

In [ ]:
#For the remaining points, compute distances to all hulls
#The iterative da has to do with teh mysterious variability in distances computed from dataframe and from single point.
outside_hulls = valX[val.assigned_species.isnull(),:]
distdf = pd.DataFrame(index=val.loc[val.assigned_species.isnull(), 'sample_id'], columns=['Anopheles_gambiae', 'Anopheles_coluzzii', 'Anopheles_arabiensis', 'Anopheles_tengrela', 'Anopheles_merus', 'Anopheles_melas', 
                                'Anopheles_quadriannulatus', 'Anopheles_bwambae-fontenillei'])
for species, hull in zip(['Anopheles_gambiae', 'Anopheles_coluzzii', 'Anopheles_arabiensis', 'Anopheles_tengrela', 'Anopheles_merus', 'Anopheles_melas', 
                                'Anopheles_quadriannulatus', 'Anopheles_bwambae-fontenillei'], 
                               [gamhull, colhull, arahull, tenhull, merhull, melhull, quadhull, bwahull]):
    da = np.zeros(outside_hulls.shape[0])
    for i in np.arange(len(da)):
        da[i] = hull_dist(hull, outside_hulls[i])
    distdf[species] = da
distdf.head()

In [ ]:
#Make summary dataframe with the distances to the closest two hulls and corresponding species
distsumdf = pd.DataFrame(index=val.loc[val.assigned_species.isnull(), 'sample_id'])
distsumdf['d1'] = distdf.min(axis=1)
distsumdf['sp1'] = distdf.idxmin(axis=1)
distsumdf['d2'] = distdf.apply(lambda x: x.sort_values()[1], axis=1)
distsumdf['sp2'] = distdf.apply(lambda x: x.sort_values().index[1], axis=1)
distsumdf.head()

In [56]:
#For gambiae-coluzzii stripe no majority rule - taken from the removed points in the convex hull trimming
gamcol = distsumdf.loc[(distsumdf.sp1.isin(['Anopheles_gambiae', 'Anopheles_coluzzii'])) 
                       & (distsumdf.sp2.isin(['Anopheles_gambiae', 'Anopheles_coluzzii']))
                      & (distsumdf.d2<14)]

In [ ]:
if gamcol.shape[0]>0:
    gcdict = dict('Uncertain_'+gamcol.sp1.str.split('_', expand=True)[1]+'_'+gamcol.sp2.str.split('_', expand=True)[1])
    val.loc[val.sample_id.isin(gamcol.index), 'assigned_species'] = val.loc[val.sample_id.isin(gamcol.index), 'sample_id'].map(gcdict)
val.groupby('assigned_species', dropna=False)['sample_id'].count()

In [ ]:
#Points that are at least 7 times closer to one convex hulls than to all others are assigned to this hull 
#to create a 'fuzzy' boundary
distsumdf.loc[(7*distsumdf.d1<distsumdf.d2) & ~(distsumdf.index.isin(gamcol.index))].groupby(['sp1', 'sp2'])['sp1'].count()

In [ ]:
sdict = dict(distsumdf.loc[7*distsumdf.d1<distsumdf.d2, 'sp1'])
val.loc[val.assigned_species.isnull(), 'assigned_species'] = val.loc[val.assigned_species.isnull(), 'sample_id'].map(sdict)
val.groupby('assigned_species', dropna=False)['sample_id'].count()

In [ ]:
distdf.loc[val.loc[val.assigned_species.isnull(), 'sample_id']].apply(lambda x: x<7*x.min(), axis=1).head()

In [ ]:
#The remaining points are assigned to all labels that are ≤ 7 times as far away as the closest hull
#In order or hull proximity
intermediates = distdf.loc[val.loc[val.assigned_species.isnull(), 'sample_id']].apply(lambda x: 'Uncertain_'+
                                    '_'.join([y.split('_')[1] for y in x.sort_values().index[x.sort_values()<7*x.min()]]), axis=1)
intermediates.head()

In [ ]:
idict = dict(intermediates)
val.loc[val.assigned_species.isnull(), 'assigned_species'] = val.loc[val.assigned_species.isnull(), 'sample_id'].map(idict)
val.groupby('assigned_species', dropna=False)['sample_id'].count()

In [63]:
val.to_csv(testdir+'gambiae_complex/assignments.tsv', index=False, sep='\t')